In [249]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import time

import warnings
warnings.filterwarnings('ignore')

In [250]:
def fetch_player_data(url):
    
    resp = requests.get(url)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text.replace('<!--', '').replace('--!>', ''), 'html.parser')

    headers, rows = [], []

    table_data = soup.find_all('tbody')[2]
    table_rows = table_data.find_all('tr')

    for value in table_rows[0]:
        headers.append(value.get('data-stat'))

    for row in table_rows:
        row_values = []
        row_data = row.find_all('td')
        for data_value in row_data:
            if data_value.get('data-append-csv'):
                row_values.append(data_value.get('data-append-csv'))
            row_values.append(data_value.get_text())
        rows.append(row_values)

    df = pd.DataFrame(rows, columns = headers).fillna(0)
    df = df.loc[df['player'] != 0].reset_index(drop = True)

    if 'minutes' in headers:
        df['minutes'] = df['minutes'].str.replace(',', '')
    elif 'gk_minutes' in headers:
        df['gk_minutes'] = df['gk_minutes'].str.replace(',', '')

    for i, row in df.iterrows():
        try:
            if '-' in row['age']:
                age_num = row['age'].split('-')[0]
                df.at[i, 'age'] = age_num
        except: pass

    df.rename(columns={'ranker' : 'player_id'}, inplace=True)
    df = df.apply(pd.to_numeric, errors = 'ignore')
    
    return df

In [251]:
def check_comp(comp):
    output = ""
    if comp.lower() == 'england' or comp.lower() == "premier_league" or comp.lower() == "eng":
        league_code = 9
        league_name = "Premier-League"
    elif comp.lower() == 'spain' or comp.lower() == "la_liga" or comp.lower() == "esp":
        league_code = 12
        league_name = "La-Liga"
    elif comp.lower() == 'italy' or comp.lower() == "serie-a" or comp.lower() == "ita":
        league_code = 11
        league_name = "Serie-A"
    elif comp.lower() == 'germany' or comp.lower() == "bundesliga" or comp.lower() == "ger":
        league_code = 20
        league_name = "Bundesliga"
    elif comp.lower() == 'france' or comp.lower() == "ligue-1" or comp.lower() == "fra":
        league_code = 13
        league_name = "Ligue-1"
    return league_code, league_name

In [252]:
def get_data(competition, year, data_type):
    
    comp = check_comp(competition)
    season = f"{str(year)}-{str(year+1)}"
    url = f"https://fbref.com/en/comps/{comp[0]}/{season}/{data_type}/{season}-{comp[1]}-Stats"
    df = fetch_player_data(url)
    
    return df

In [258]:
df = get_data("ita", 2023, "keepers")

In [260]:
df.head(5)

,player_id,player,nationality,position,team,age,birth_year,gk_games,gk_games_starts,gk_minutes,...,gk_ties,gk_losses,gk_clean_sheets,gk_clean_sheets_pct,gk_pens_att,gk_pens_allowed,gk_pens_saved,gk_pens_missed,gk_pens_save_pct,matches
0,a9eaccda,Simone Aresti,it ITA,GK,Cagliari,37,1986,1,0,1,...,0,0,0,NaN,1,1,0,0,0.0,Matches
1,649b01d8,Emil Audero,it ITA,GK,Inter,26,1997,4,4,337,...,1,1,2,50.0,0,0,0,0,NaN,Matches
2,ad263695,Nicola Bagnolini,it ITA,GK,Bologna,19,2004,1,0,5,...,0,0,0,NaN,0,0,0,0,NaN,Matches
3,ee833569,Etrit Berisha,al ALB,GK,Empoli,34,1989,14,14,1260,...,3,8,4,28.6,3,2,1,0,33.3,Matches
4,97d7afbd,Elia Caprile,it ITA,GK,Empoli,21,2001,23,23,2070,...,6,11,5,21.7,5,4,1,0,20.0,Matches


In [202]:
## normal player stats work for all leagues and players

## need to fix columns problem with other leagues
## need to figure out how to change minutes from objects to floats without hard coding